In [ ]:
!pip install -q torch datasets accelerate bitsandbytes scikit-learn evaluate rouge_score
# evaluate, rouge_score weren't used later due to lack of GPU memory on Kaggle (more than 16GB used) 
!pip install -q --upgrade transformers datasets tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account 

In [2]:
from transformers import logging
logging.set_verbosity_info()

from transformers.utils.logging import enable_progress_bar
enable_progress_bar()

import datasets
datasets.disable_progress_bar()

In [3]:
import os
import random
import json
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import torch

print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# create working directories
os.makedirs("/kaggle/working/data", exist_ok=True)

CUDA available: True
Device: cuda


In [4]:
# defining templates for enrichment
templates = [
    "preprocess: {sent} 😊",
    "preprocess: 🎉 {sent} Here’s why it matters…",
    "preprocess: 💡 {sent} What I took away…",
    "preprocess: 🔍 {sent} Key point:",
    "preprocess: 🚀 {sent} Ready to take off!",
    "preprocess: 📣 {sent} Spread the word!",
    "preprocess: 📝 Quick recap: {sent}",
    "preprocess: 🎯 Key takeaway: {sent}",
    "preprocess: 🔥 {sent} My top 3 insights…",
    "preprocess: 🌟 {sent} Shine bright today!",
    "preprocess: 💬 {sent} Would love your thoughts.",
    "preprocess: 🤔 {sent} What do you think?",
    "preprocess: ✨ {sent} Here’s what stood out…",
    "preprocess: 🌱 {sent} Growth moment:",
    "preprocess: 🎙️ {sent} Here’s the story…",
    "preprocess: ✅ {sent} Lesson learned.",
    "preprocess: 💭 Reflecting: {sent}",
    "preprocess: 📈 {sent} Trends update!",
    "preprocess: 🛠️ {sent} How to apply:",
    "preprocess: 💼 {sent} Business angle:",
    "preprocess: 🌐 {sent} Global perspective:",
    "preprocess: 💎 {sent} Gem of insight:",
    "preprocess: 🏆 {sent} Achievement unlocked:",
    "preprocess: 📊 {sent} Data speaks:",
    "preprocess: 🎥 {sent} Watch now:",
    "preprocess: 📚 {sent} Key learnings:",
    "preprocess: 🤝 {sent} Let’s collaborate!",
    "preprocess: 🔗 {sent} Check the link below:",
    "preprocess: 🕒 {sent} Time check:",
    "preprocess: 📅 {sent} Save the date:",
    "preprocess: 🔄 {sent} Updated insights:",
    "preprocess: 🧠 {sent} Food for thought:",
    "preprocess: 🌟 {sent} Spotlight:",
    "preprocess: 🌱 {sent} Growth hack:",
    "preprocess: 🎯 {sent} Laser focus:",
    "preprocess: 🔍 {sent} Deep dive:",
    "preprocess: {sent} Here's what I learned.",
    "preprocess: Reflecting on {sent}, I've noted several insights.",
    "preprocess: Recently, {sent}. Let me explain further.",
    "preprocess: Delighted to share that {sent}.",
    "preprocess: {sent} Key takeaways below.",
    "preprocess: Had the opportunity to {sent}; here are my reflections.",
    "preprocess: {sent}. Here's why it's significant professionally.",
    "preprocess: An insightful experience: {sent}.",
    "preprocess: {sent} Here are the core ideas.",
    "preprocess: Exploring the topic of {sent}; here's my perspective.",
    "preprocess: I recently encountered {sent}. Here’s what stood out.",
    "preprocess: Sharing thoughts on {sent}.",
    "preprocess: A professional milestone: {sent}.",
    "preprocess: {sent} Let’s discuss the implications.",
    "preprocess: Recently focused on {sent}. Here are valuable insights.",
    "preprocess: {sent} Here are strategies that worked.",
    "preprocess: {sent} This matters because:",
    "preprocess: {sent} Highlighting key points:",
    "preprocess: {sent} Professional reflections follow.",
    "preprocess: Had an insightful discussion about {sent}. Here’s what emerged.",
    "preprocess: Attended an insightful session on {sent}. My main takeaways:",
    "preprocess: Recently presented about {sent}; here are some essential points.",
    "preprocess: Sharing professional learnings from {sent}.",
    "preprocess: {sent} Why does it matter professionally?",
    "preprocess: Gained clarity around {sent}; here’s how.",
    "preprocess: Key professional insights gained from {sent}:",
    "preprocess: A valuable perspective on {sent}:",
    "preprocess: Thought leadership insight: {sent}",
    "preprocess: Recently engaged in {sent}. Professional lessons learned:",
    "preprocess: Professional insight into {sent}:",
    "preprocess: Exploring implications of {sent}.",
    "preprocess: {sent} Critical professional considerations:",
    "preprocess: Discussing the importance of {sent}.",
    "preprocess: Key observations regarding {sent}:",
    "preprocess: Sharing experiences about {sent}:",
    "preprocess: My professional perspective on {sent}:",
    "preprocess: {sent} Essential reflections:",
    "preprocess: Addressing the professional relevance of {sent}.",
    "preprocess: {sent} Here’s a summary of the key insights.",
    "preprocess: Key professional learnings from {sent}.",
    "preprocess: Important discussion points regarding {sent}:",
    "preprocess: Strategic insights from {sent}:",
    "preprocess: {sent} What professionals should consider:",
    "preprocess: Sharing a professional viewpoint on {sent}.",
    "preprocess: Professional insights derived from {sent}.",
    "preprocess: Analyzing the professional impact of {sent}.",
    "preprocess: Highlighting important outcomes from {sent}.",
    "preprocess: Professional reflection on {sent}.",
    "preprocess: My professional takeaways from {sent}.",
    "preprocess: Key insights from my recent experience with {sent}.",
    "preprocess: Important findings about {sent}.",
    "preprocess: Professional review of {sent}.",
    "preprocess: Sharing detailed insights about {sent}.",
    "preprocess: Professional thoughts on {sent}:",
    "preprocess: Insightful outcomes from {sent}.",
    "preprocess: Professional perspective following {sent}:",
    "preprocess: Reflecting professionally on {sent}.",
    "preprocess: Discussing professional experiences with {sent}.",
    "preprocess: Essential lessons from {sent}.",
    "preprocess: Professional commentary on {sent}.",
    "preprocess: Evaluating the professional significance of {sent}.",
    "preprocess: Key reflections about {sent}.",
    "preprocess: Strategic professional insights from {sent}.",
    "preprocess: Professional development lessons from {sent}.",
    "preprocess: Sharing a professional summary of {sent}.",
    "preprocess: Professional insights: {sent}.",
    "preprocess: My recent experience with {sent}; important outcomes:",
    "preprocess: Professional analysis of {sent}.",
    "preprocess: Reflecting on professional growth via {sent}.",
    "preprocess: Insights gained professionally from {sent}.",
    "preprocess: Professional experience highlights: {sent}.",
    "preprocess: Discussing the professional implications of {sent}.",
    "preprocess: Professional considerations from {sent}.",
    "preprocess: A professional review of {sent}.",
    "preprocess: Important professional takeaways from {sent}.",
    "preprocess: Highlighting professional observations about {sent}.",
    "preprocess: Professional insights on {sent}.",
    "preprocess: Thought-provoking professional findings from {sent}.",
    "preprocess: Strategic considerations regarding {sent}.",
    "preprocess: My professional reflection on {sent}.",
    "preprocess: Detailed professional insights from {sent}.",
    "preprocess: A professional reflection about {sent}.",
    "preprocess: Insightful professional lessons from {sent}.",
    "preprocess: Discussing key professional outcomes from {sent}.",
    "preprocess: Critical professional insights: {sent}.",
    "preprocess: Strategic professional perspective on {sent}.",
    "preprocess: Essential professional insights about {sent}.",
    "preprocess: My analysis and insights from {sent}.",
    "preprocess: Professional reflections regarding {sent}.",
    "preprocess: Examining professional learnings from {sent}.",
    "preprocess: Discussing the professional takeaways from {sent}.",
    "preprocess: Sharing important professional learnings from {sent}.",
    "preprocess: Summarizing my professional experience with {sent}.",
    "preprocess: Valuable professional insights regarding {sent}.",
    "preprocess: A reflection on professional growth through {sent}.",
    "preprocess: Analyzing professional implications of {sent}.",
    "preprocess: Professional perspectives gained from {sent}.",
    "preprocess: Sharing insights from my recent professional experience: {sent}.",
    "preprocess: Insights and reflections about {sent} from a professional viewpoint.",
    "preprocess: Discussing the professional insights gained from {sent}."
]

In [5]:
# loading CommonGen train+validation split
cg = load_dataset("common_gen", split="train+validation")

# generating pairs
pairs = []
for ex in cg:
    raw = ", ".join(ex["concepts"])
    sent = ex["target"]
    tmpl = random.choice(templates)
    refined = tmpl.format(sent=sent)
    pairs.append({"raw": raw, "refined": refined})

# splitting into train/val/test (80/10/10)
train_val, test = train_test_split(pairs, test_size=0.10, random_state=42)
train, val    = train_test_split(train_val, test_size=0.11, random_state=42)  # ~10% of original

# saving to JSONL
for split, data in (("train", train), ("validation", val), ("test", test)):
    path = f"/kaggle/working/data/{split}.jsonl"
    with open(path, "w") as f:
        for ex in data:
            f.write(json.dumps(ex, ensure_ascii=False) + "\n")
    print(f"Saved {split} → {path}")

README.md:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/171k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/31.2k [00:00<?, ?B/s]

Saved train → /kaggle/working/data/train.jsonl
Saved validation → /kaggle/working/data/validation.jsonl
Saved test → /kaggle/working/data/test.jsonl


In [14]:
from transformers import (
    T5Tokenizer, T5ForConditionalGeneration,
    Trainer, TrainingArguments, EarlyStoppingCallback
)
from datasets import load_dataset

# loading JSONL datasets
data_files = {
    "train": "/kaggle/working/data/train.jsonl",
    "validation": "/kaggle/working/data/validation.jsonl"
}
ds = load_dataset("json", data_files=data_files)

# initializing tokenizer & model
model_name = "t5-small"
tokenizer  = T5Tokenizer.from_pretrained(model_name)
model      = T5ForConditionalGeneration.from_pretrained(model_name)

# preprocessing function
def preprocess_fn(ex):
    inp = "preprocess: " + ex["raw"]
    tgt = ex["refined"]
    mi  = tokenizer(inp,  max_length=64, truncation=True, padding="max_length")
    lab = tokenizer(tgt,  max_length=64, truncation=True, padding="max_length")
    mi["labels"] = lab["input_ids"]
    return mi

tokenized = ds.map(preprocess_fn, batched=False, remove_columns=["raw","refined"])

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/tokenizer_config.json
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/tokenizer.json
loading file chat_template.jinja from cache at None
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/config.json
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "clas

In [19]:
training_args = TrainingArguments(
    output_dir="/kaggle/working/slm_preproc_ckpt",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    eval_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    save_steps=2000,
    fp16=True, # GPU-enabled on Kaggle Jupyter Notebook
    disable_tqdm=False,
    report_to=["none"],
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    processing_class=tokenizer,
)

trainer.train()
trainer.save_model("/kaggle/working/slm_preproc_final")


PyTorch: setting up devices
Using auto half precision backend
***** Running training *****
  Num examples = 57,196
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 17,880
  Number of trainable parameters = 60,506,624


Step,Training Loss,Validation Loss
500,1.102200,0.957670
1000,1.037300,0.884950
1500,0.959200,0.828353
2000,0.908900,0.788100
2500,0.875400,0.759752
3000,0.845400,0.739392
3500,0.824000,0.724401
4000,0.810100,0.713164
4500,0.794800,0.704986
5000,0.781900,0.698115



***** Running Evaluation *****
  Num examples = 7070
  Batch size = 16

***** Running Evaluation *****
  Num examples = 7070
  Batch size = 16

***** Running Evaluation *****
  Num examples = 7070
  Batch size = 16

***** Running Evaluation *****
  Num examples = 7070
  Batch size = 16
Saving model checkpoint to /kaggle/working/slm_preproc_ckpt/checkpoint-2000
Configuration saved in /kaggle/working/slm_preproc_ckpt/checkpoint-2000/config.json
Configuration saved in /kaggle/working/slm_preproc_ckpt/checkpoint-2000/generation_config.json
Model weights saved in /kaggle/working/slm_preproc_ckpt/checkpoint-2000/model.safetensors
tokenizer config file saved in /kaggle/working/slm_preproc_ckpt/checkpoint-2000/tokenizer_config.json
Special tokens file saved in /kaggle/working/slm_preproc_ckpt/checkpoint-2000/special_tokens_map.json
added tokens file saved in /kaggle/working/slm_preproc_ckpt/checkpoint-2000/added_tokens.json

***** Running Evaluation *****
  Num examples = 7070
  Batch size = 

In [20]:
from transformers import pipeline

pipe = pipeline(
    "text2text-generation",
    model="/kaggle/working/slm_preproc_final",
    tokenizer="/kaggle/working/slm_preproc_final",
    device=0  # GPU
)

for raw in [
    "branding, college, talk",
    "launch, product, next week",
    "sustainable design, workshop, attendees"
]:
    out = pipe("preprocess: " + raw, max_length=64)[0]["generated_text"]
    print(f"> RAW: {raw}\n→ {out}\n")


loading configuration file /kaggle/working/slm_preproc_final/config.json
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
     

> RAW: branding, college, talk
→ preprocess: Discussing the professional takeaways from students talk about branding at a college ..



Both `max_new_tokens` (=256) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


> RAW: launch, product, next week
→ preprocess: Discussing the professional takeaways from the launch of the product next week ..

> RAW: sustainable design, workshop, attendees
→ preprocess: Discussing the professional insights gained from attendees attended a workshop on sustainable design ..

